In [1]:
import mucus_rust as mc
import matplotlib.pyplot as plt
import numpy as np

import h5py
import os

In [20]:
cfg_dir = "/net/data04/janmak98/results/big_cleanup/new_lj/longloop/notracer/configs"
cfg_dir = "/net/data04/janmak98/results/january/final/5tracer5a/configs"

cfg_relpaths = os.listdir(cfg_dir)
cfg_paths = [cfg_dir + "/" + cfg_fname for cfg_fname in cfg_relpaths]
for i, cfg_path in enumerate(cfg_paths):
    print(f'{i:2d}', cfg_path)
    
    
idx_cfg = 1
fname_cfg = cfg_paths[idx_cfg]
print("\nSelected system:\n", idx_cfg, fname_cfg)

 0 /net/data04/janmak98/results/january/final/5tracer5a/configs/cfg_long_loop_3wtp.toml
 1 /net/data04/janmak98/results/january/final/5tracer5a/configs/cfg_long_loop_2wtp.toml
 2 /net/data04/janmak98/results/january/final/5tracer5a/configs/cfg_long_loop_4wtp.toml
 3 /net/data04/janmak98/results/january/final/5tracer5a/configs/cfg_long_loop_1wtp.toml

Selected system:
 1 /net/data04/janmak98/results/january/final/5tracer5a/configs/cfg_long_loop_2wtp.toml


In [21]:

#! CAREFUL CAREFUL CAREFUL CAREFUL CAREFUL CAREFUL CAREFUL CAREFUL CAREFUL CAREFUL CAREFUL CAREFUL
#! DO NOT LOAD EVERYTHIN INTO MEMEORY IF THE TRAJECTORY IS LONG

cfg = mc.Config.from_toml(fname_cfg)

n_frames_tot = mc.utils.get_number_of_frames(cfg)
print("Total number of frames:", n_frames_tot)

#! CHOOSE FRAME RANGE AND STRIDE
frame_range = [0, 100]
stride = 1

frame_range = mc.utils._validate_frame_range(frame_range, n_frames_tot)

print("Loading frames", frame_range[0], "-", frame_range[1], "with stride", stride)

Total number of frames: 1000000
Loading frames 0 - 100 with stride 1


In [22]:
cfg = mc.Config.from_toml(fname_cfg)

# get dataset names
dset_traj = mc.utils.ResultsFiletypes.Trajectory.value
dset_forc = mc.utils.ResultsFiletypes.Forces.value
dset_dist = mc.utils.ResultsFiletypes.Distances.value

#trajectory = h5py.File(mc.utils.get_path(cfg, dset_traj), "r", locking='false')[dset_traj][frame_range[0]:frame_range[1]:stride]
#forces     = h5py.File(mc.utils.get_path(cfg, dset_forc), "r", locking='false')[dset_forc][frame_range[0]:frame_range[1]:stride]
distances  = h5py.File(mc.utils.get_path(cfg, dset_dist), "r", locking='false')[dset_dist][frame_range[0]:frame_range[1]:stride]

sim = mc.Analysis(cfg=cfg, frame_range=None, stride=1)

In [27]:
#off_diag_dist = distances[:][[np.triu_indices(cfg.n_particles, k = 1)]]

for i in range(1, 100):
    if np.any(distances[i][np.triu_indices(cfg.n_particles, k = 1)] == 0):
        print(i)
        print(distances[i])
        print(np.sum(distances[i][np.triu_indices(cfg.n_particles, k = 1)] == 0))
        print(cfg.n_particles**2)
        print(distances[i].shape)
        print(distances[i][np.triu_indices(cfg.n_particles, k = 1)])
        break

[False False False ... False False False]
1
[[ 0.     2.     2.828 ... 17.8   10.71   9.12 ]
 [ 2.     0.     2.    ... 18.84  11.85   7.145]
 [ 2.828  2.     0.    ... 18.52  10.39   7.727]
 ...
 [17.8   18.84  18.52  ...  0.    16.69  21.75 ]
 [10.71  11.85  10.39  ... 16.69   0.    17.8  ]
 [ 9.12   7.145  7.727 ... 21.75  17.8    0.   ]]
2
69169
(263, 263)
[ 2.     2.828  4.473 ... 16.69  21.75  17.8  ]


In [29]:
print(cfg.n_particles)
for i in range(cfg.n_particles-1):
    for j in range(i+1, cfg.n_particles):
        d = distances[1][i,j]
        if d == 0:
            print(i,j)

263
0 128
129 257


In [26]:
A = np.ones([5, 5])
A[np.triu_indices(5, k = 1)] = 0
print(A)


[[1. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0.]
 [1. 1. 1. 0. 0.]
 [1. 1. 1. 1. 0.]
 [1. 1. 1. 1. 1.]]


In [24]:
n_krise = 0
for d in off_diag_dist:
    if d == 0:
        n_krise += 1
        
        
print(n_krise)

NameError: name 'off_diag_dist' is not defined